# Learning Tic-Tac-Toe with Reinforcement Learning
**_Train with SageMaker RL and evaluate interactively within the notebook_**

---

---


## Outline

1. [Overview](#Overview)
1. [Setup](#Setup)
1. [Code](#Code)
  1. [Environment](#Environment)
  1. [Preset](#Preset)
  1. [Launcher](#Launcher)
1. [Train](#Train)
1. [Deploy](#Deploy)
  1. [Inference](#Inference)
1. [Play](#Play)
1. [Wrap Up](#Wrap-Up)

---

## Overview

Tic-tac-toe is one of the first games children learn to play and was one of the [first computer games ever](https://en.wikipedia.org/wiki/OXO).  Optimal play through exhaustive search is relatively straightforward, however, approaching with a reinforcement learning agent can be educational.

This notebook shows how to train a reinforcement learning agent with SageMaker RL and then play locally and interactively within the notebook.  Unlike SageMaker local mode, this method does not require a docker container to run locally, instead using an endpoint and integration with a small Jupyter app.

---

## Setup

Let's start by defining our S3 bucket and and IAM role.

In [1]:
import sagemaker

bucket = sagemaker.Session().default_bucket()
role = sagemaker.get_execution_role()

Let's import the libraries we'll use.

In [27]:
import os
import numpy as np
import sagemaker
from sagemaker.rl import RLEstimator, RLToolkit, RLFramework
from tic_tac_toe_game import TicTacToeGame

---

## Code

Our tic-tac-toe example requires 3 scripts in order to train our agent using SageMaker RL.  The scripts are placed in the `./src` directory which is sent to the container when the SageMaker training job is initiated.

### Environment

For our tic-tac-toe use case we'll create a custom Gym environment.  This means we'll specify a Python class which inherits from `gym.Env` and has two methods: `reset()` and `step()`.  These will provide the agent its state, actions, and rewards for learning.  In more detail:

The `__init__()` method is called at the beginning of the SageMaker training job and:
1. Starts the 3x3 tic-tac-toe board as a NumPy array of zeros
1. Prepares the state space as a flattened version of the board (length 9)
1. Defines a discrete action space with 9 possible options (one for each place on the board)

The `reset()` method is called at the beginning of each episode and:
1. Clears the 3x3 board (sets all values to 0)
1. Does some minor record-keeping for tracking across tic-tac-toe games

The `step()` method is called for each iteration in an episode and:
1. Adjusts the board based on the action chosen by the agent based on the previous state
1. Generates rewards based on performance
1. Automatically chooses the move for the agent's opponent if needed

Note:
* The opponent has not been programmed for perfect play.  If we taught our agent against a perfect opponent, it would not generalize to scenarios where the rules of perfect play were not followed.
* If our agent selects an occupied space, it is given a minor penalty (-0.1) and asked to choose again.  Although the state doesn't change across these steps (meaning the agent's network's prediction should stay the same), randomness in the agent should eventually result in different actions.  However, if the agent chooses an occupied space 10 times in a row, the game is forfeit.  Selecting an action only from available spaces would require more substantial modification than was desired for this example.
* Other rewards only occur when a game is completed (+1 for win, 0 for draw, -1 for loss).
* The board is saved as a NumPy array where a value of +1 represents our agent's moves (`X`s) and a value of -1 represents the opponent's moves (`O`s).

In [3]:
!pygmentize ./src/tic_tac_toe.py

import gym
from gym import spaces
import numpy as np
import os
import time


class TicTacToeEnv(gym.Env):


    def __init__(self, opponent='moderate'):
        self.opponent = opponent
        self.episode = 0
        self.observation_space = spaces.Box(low=-1, high=1, shape=(9, ), dtype=np.int)
        self.action_space = spaces.Discrete(9)


    def reset(self):
        self.episode += 1
        self.total_reward = 0
        self.occupied = 0
        self.board = np.zeros((3, 3))
        state = self.board.flatten()
        return state


    def step(self, action):

        # Convert action into board position
        row = action // 3
        col = action % 3

        # If agent picks an occupied space repeated end the game and give a penalty
        # Otherwise, give a small penalty and try again
        if self.board[row, col] != 0:
            self.occupied += 1
            if self.occupied > 10:
                reward = -1
                self.total_reward += reward
          

### Preset

The preset file specifies Coach parameters used by our reinforcement learning agent.  For this problem we'll use a [Clipped PPO algorithm](https://nervanasystems.github.io/coach/components/agents/policy_optimization/cppo.html).  We have kept the preset file deliberately spartan, deferring to defaults for most parameters, in order to focus on just the key components.  Performance of our agent could likely be improved with increased tuning.

In [4]:
!pygmentize ./src/preset.py

from rl_coach.agents.clipped_ppo_agent import ClippedPPOAgentParameters
from rl_coach.base_parameters import VisualizationParameters, PresetValidationParameters
from rl_coach.core_types import TrainingSteps, EnvironmentEpisodes, EnvironmentSteps
from rl_coach.environments.gym_environment import GymVectorEnvironment
from rl_coach.graph_managers.basic_rl_graph_manager import BasicRLGraphManager
from rl_coach.graph_managers.graph_manager import ScheduleParameters

####################
# Graph Scheduling #
####################

schedule_params = ScheduleParameters()
schedule_params.improve_steps = TrainingSteps(50000)
schedule_params.steps_between_evaluation_periods = EnvironmentSteps(2000)
schedule_params.evaluation_steps = EnvironmentEpisodes(5)
schedule_params.heatup_steps = EnvironmentSteps(0)

#########
# Agent #
#########

agent_params = ClippedPPOAgentParameters()

###############
# Environment #
###############

env_params = GymVectorEnvironment(level='tic_tac_toe:TicTacToeEnv')

#

### Launcher

The launcher is a script used by Amazon SageMaker to drive the training job on the SageMaker RL container.  We have kept it minimal, only specifying the name of the preset file to be used for the training job.

In [5]:
!pygmentize ./src/train-coach.py

from sagemaker_rl.coach_launcher import SageMakerCoachPresetLauncher


class MyLauncher(SageMakerCoachPresetLauncher):

    def default_preset_name(self):
        """This points to a .py file that configures everything about the RL job.
        It can be overridden at runtime by specifying the RLCOACH_PRESET hyperparameter.
        """
        return 'preset'


if __name__ == '__main__':
    MyLauncher.train_main()


---

## Train

Now, let's kick off the training job in Amazon SageMaker.  This call can include hyperparameters that overwrite values in `train-coach.py` or `preset.py`, but in our case, we've limited to defining:
1. The location of our agent code `./src` and dependencies in `common`.
1. Which RL and DL framework to use (SageMaker also supports [Ray RLlib](https://ray.readthedocs.io/en/latest/rllib.html) and Coach TensorFlow).
1. The IAM role granted permissions to our data in S3 and ability to create SageMaker training jobs.
1. Training job hardware specifications (in this case just 1 ml.m4.xlarge instance).
1. Output path for our checkpoints and saved episodes.
1. A single hyperparameter specifying that we would like our agent's network to be output (in this case as an ONNX model).

In [6]:
estimator = RLEstimator(source_dir='src',
                        entry_point="train-coach.py",
                        dependencies=["common/sagemaker_rl"],
                        toolkit=RLToolkit.COACH,
                        toolkit_version='0.11.0',
                        framework=RLFramework.MXNET,
                        role=role,
                        train_instance_count=1,
                        train_instance_type='ml.m4.xlarge',
                        output_path='s3://{}/'.format(bucket),
                        base_job_name='DEMO-rl-tic-tac-toe',
                        hyperparameters={'save_model': 1})

estimator.fit()

2019-06-07 15:35:19 Starting - Starting the training job...
2019-06-07 15:35:21 Starting - Launching requested ML instances.........
2019-06-07 15:36:53 Starting - Preparing the instances for training......
2019-06-07 15:38:10 Downloading - Downloading input data
2019-06-07 15:38:10 Training - Downloading the training image..
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-06-07 15:38:30,105 sagemaker-containers INFO     Imported framework sagemaker_mxnet_container.training
2019-06-07 15:38:30,109 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-06-07 15:38:30,123 sagemaker_mxnet_container.training INFO     MXNet training environment: {'SM_TRAINING_ENV': '{"additional_framework_parameters":{"sagemaker_estimator":"RLEstimator"},"channel_input_dirs":{},"current_host":"algo-1","framework_module":"sagemaker_mxnet_container.training:main","hosts":["algo-1"],"hyperparameters":{"save_model":1

Training> Name=main_level/agent, Worker=0, Episode=1, Total reward=-1.1, Steps=5, Training iteration=0
Training> Name=main_level/agent, Worker=0, Episode=2, Total reward=-1, Steps=8, Training iteration=0
Training> Name=main_level/agent, Worker=0, Episode=3, Total reward=-1.2, Steps=13, Training iteration=0
Training> Name=main_level/agent, Worker=0, Episode=4, Total reward=-1.1, Steps=17, Training iteration=0
Training> Name=main_level/agent, Worker=0, Episode=5, Total reward=-0.2, Steps=24, Training iteration=0
Training> Name=main_level/agent, Worker=0, Episode=6, Total reward=-0.5, Steps=34, Training iteration=0
Training> Name=main_level/agent, Worker=0, Episode=7, Total reward=-0.7, Steps=46, Training iteration=0
Training> Name=main_level/agent, Worker=0, Episode=8, Total reward=-1.1, Steps=51, Training iteration=0
Training> Name=main_level/agent, Worker=0, Episode=9, Total reward=0.1, Steps=64, Training iteration=0
Training> Name=main_level/agent, Worker=0, Episode=10, Total reward=-

Training> Name=main_level/agent, Worker=0, Episode=173, Total reward=-1, Steps=1242, Training iteration=0
Training> Name=main_level/agent, Worker=0, Episode=174, Total reward=-0.6, Steps=1253, Training iteration=0
Training> Name=main_level/agent, Worker=0, Episode=175, Total reward=-1, Steps=1257, Training iteration=0
Training> Name=main_level/agent, Worker=0, Episode=176, Total reward=-2.1, Steps=1272, Training iteration=0
Training> Name=main_level/agent, Worker=0, Episode=177, Total reward=-0.4, Steps=1281, Training iteration=0
Training> Name=main_level/agent, Worker=0, Episode=178, Total reward=-1, Steps=1284, Training iteration=0
Training> Name=main_level/agent, Worker=0, Episode=179, Total reward=-1, Steps=1287, Training iteration=0
Training> Name=main_level/agent, Worker=0, Episode=180, Total reward=-2.1, Steps=1303, Training iteration=0
Training> Name=main_level/agent, Worker=0, Episode=181, Total reward=-1, Steps=1306, Training iteration=0
Training> Name=main_level/agent, Worke

Policy training> Surrogate loss=-0.016231467947363853, KL divergence=[0.], Entropy=[-0.0218709], training epoch=6, learning_rate=0.00025
Policy training> Surrogate loss=-0.01799607463181019, KL divergence=[0.], Entropy=[-0.02186245], training epoch=7, learning_rate=0.00025
Policy training> Surrogate loss=-0.018739134073257446, KL divergence=[0.], Entropy=[-0.02184578], training epoch=8, learning_rate=0.00025
Policy training> Surrogate loss=-0.01849059946835041, KL divergence=[0.], Entropy=[-0.02183821], training epoch=9, learning_rate=0.00025
Checkpoint> Saving in path=['/opt/ml/output/data/checkpoint/1_Step-2053.ckpt.main_level.agent.main.online', '/opt/ml/output/data/checkpoint/1_Step-2053.ckpt.main_level.agent.main.online.onnx']
Training> Name=main_level/agent, Worker=0, Episode=280, Total reward=-1.3, Steps=2060, Training iteration=1
Training> Name=main_level/agent, Worker=0, Episode=281, Total reward=-0.8, Steps=2073, Training iteration=1
Training> Name=main_level/agent, Worker=0,


2019-06-07 15:39:01 Training - Training image download completed. Training in progress.Training> Name=main_level/agent, Worker=0, Episode=411, Total reward=-1.3, Steps=3095, Training iteration=1
Training> Name=main_level/agent, Worker=0, Episode=412, Total reward=-1, Steps=3099, Training iteration=1
Training> Name=main_level/agent, Worker=0, Episode=413, Total reward=-1.1, Steps=3103, Training iteration=1
Training> Name=main_level/agent, Worker=0, Episode=414, Total reward=-0.7, Steps=3115, Training iteration=1
Training> Name=main_level/agent, Worker=0, Episode=415, Total reward=-1, Steps=3118, Training iteration=1
Training> Name=main_level/agent, Worker=0, Episode=416, Total reward=-0.6, Steps=3129, Training iteration=1
Training> Name=main_level/agent, Worker=0, Episode=417, Total reward=-1.4, Steps=3136, Training iteration=1
Training> Name=main_level/agent, Worker=0, Episode=418, Total reward=-1.1, Steps=3140, Training iteration=1
Training> Name=main_level/agent, Worker=0, Episode=4

...

Training> Name=main_level/agent, Worker=0, Episode=5666, Total reward=-1.1, Steps=46381, Training iteration=22
Training> Name=main_level/agent, Worker=0, Episode=5667, Total reward=-0.9, Steps=46395, Training iteration=22
Training> Name=main_level/agent, Worker=0, Episode=5668, Total reward=-3.1, Steps=46421, Training iteration=22
Training> Name=main_level/agent, Worker=0, Episode=5669, Total reward=-1.2, Steps=46426, Training iteration=22
Training> Name=main_level/agent, Worker=0, Episode=5670, Total reward=-1.5, Steps=46435, Training iteration=22
Training> Name=main_level/agent, Worker=0, Episode=5671, Total reward=-2.1, Steps=46451, Training iteration=22
Training> Name=main_level/agent, Worker=0, Episode=5672, Total reward=-1.1, Steps=46455, Training iteration=22
Training> Name=main_level/agent, Worker=0, Episode=5673, Total reward=-2.0, Steps=46470, Training iteration=22
Training> Name=main_level/agent, Worker=0, Episode=5674, Total reward=0.2, Steps=46482, Training iteration=22
Tr

Policy training> Surrogate loss=-0.013428492471575737, KL divergence=[0.], Entropy=[-0.01978944], training epoch=4, learning_rate=0.00025
Policy training> Surrogate loss=-0.013824593275785446, KL divergence=[0.], Entropy=[-0.01980574], training epoch=5, learning_rate=0.00025
Policy training> Surrogate loss=-0.01403997465968132, KL divergence=[0.], Entropy=[-0.0197537], training epoch=6, learning_rate=0.00025
Policy training> Surrogate loss=-0.014880860224366188, KL divergence=[0.], Entropy=[-0.01974027], training epoch=7, learning_rate=0.00025
Policy training> Surrogate loss=-0.013231315650045872, KL divergence=[0.], Entropy=[-0.01975147], training epoch=8, learning_rate=0.00025
Policy training> Surrogate loss=-0.013218604028224945, KL divergence=[0.], Entropy=[-0.01968562], training epoch=9, learning_rate=0.00025
Checkpoint> Saving in path=['/opt/ml/output/data/checkpoint/45_Step-47229.ckpt.main_level.agent.main.online', '/opt/ml/output/data/checkpoint/45_Step-47229.ckpt.main_level.ag

Testing> Name=main_level/agent, Worker=0, Episode=5861, Total reward=-2.0, Steps=48000, Training iteration=23
Testing> Name=main_level/agent, Worker=0, Episode=5861, Total reward=-2.0, Steps=48000, Training iteration=23
Testing> Name=main_level/agent, Worker=0, Episode=5861, Total reward=1, Steps=48000, Training iteration=23
## agent: Finished evaluation phase. Success rate = 0.0, Avg Total Reward = -1.4
Training> Name=main_level/agent, Worker=0, Episode=5862, Total reward=-0.6, Steps=48011, Training iteration=23
Training> Name=main_level/agent, Worker=0, Episode=5863, Total reward=-1.3, Steps=48017, Training iteration=23
Training> Name=main_level/agent, Worker=0, Episode=5864, Total reward=-1, Steps=48020, Training iteration=23
Training> Name=main_level/agent, Worker=0, Episode=5865, Total reward=-0.9, Steps=48034, Training iteration=23
Training> Name=main_level/agent, Worker=0, Episode=5866, Total reward=-1.4, Steps=48042, Training iteration=23
Training> Name=main_level/agent, Worker

Training> Name=main_level/agent, Worker=0, Episode=5995, Total reward=-0.1, Steps=49163, Training iteration=23
Training> Name=main_level/agent, Worker=0, Episode=5996, Total reward=-1.2, Steps=49168, Training iteration=23
Training> Name=main_level/agent, Worker=0, Episode=5997, Total reward=-0.1, Steps=49174, Training iteration=23
Training> Name=main_level/agent, Worker=0, Episode=5998, Total reward=-1.5, Steps=49182, Training iteration=23
Training> Name=main_level/agent, Worker=0, Episode=5999, Total reward=-2.0, Steps=49197, Training iteration=23
Training> Name=main_level/agent, Worker=0, Episode=6000, Total reward=-1.1, Steps=49201, Training iteration=23
Training> Name=main_level/agent, Worker=0, Episode=6001, Total reward=-1, Steps=49205, Training iteration=23
Training> Name=main_level/agent, Worker=0, Episode=6002, Total reward=-2.3, Steps=49223, Training iteration=23
Training> Name=main_level/agent, Worker=0, Episode=6003, Total reward=-1.1, Steps=49227, Training iteration=23
Tra


2019-06-07 15:47:46 Uploading - Uploading generated training modelTraining> Name=main_level/agent, Worker=0, Episode=6052, Total reward=0.1, Steps=49673, Training iteration=24
Training> Name=main_level/agent, Worker=0, Episode=6053, Total reward=-2.1, Steps=49689, Training iteration=24
Training> Name=main_level/agent, Worker=0, Episode=6054, Total reward=-1.1, Steps=49705, Training iteration=24
Training> Name=main_level/agent, Worker=0, Episode=6055, Total reward=-1.3, Steps=49712, Training iteration=24
Training> Name=main_level/agent, Worker=0, Episode=6056, Total reward=-1.2, Steps=49718, Training iteration=24
Training> Name=main_level/agent, Worker=0, Episode=6057, Total reward=-0.9, Steps=49732, Training iteration=24
Training> Name=main_level/agent, Worker=0, Episode=6058, Total reward=-0.6, Steps=49743, Training iteration=24
Training> Name=main_level/agent, Worker=0, Episode=6059, Total reward=-1.1, Steps=49748, Training iteration=24
Training> Name=main_level/agent, Worker=0, Epi

---

## Deploy

Normally we would evaluate our agent by looking for reward convergence or monitoring performance across epsisodes.  Other SageMaker RL example notebooks cover this in detail.  We'll skip that for the more tangible approach of testing the trained agent by playing against it ourselves.  To do that, we'll first deploy the agent to a realtime endpoint to get predictions.

### Inference

Our deployment code:
1. Unpacks the ONNX model output and prepares it for inference in `model_fn`
1. Generates predictions from our network, given state (a flattened tic-tac-toe board) in `transform_fn`

In [7]:
!pygmentize ./src/deploy-coach.py

import os
import json
import mxnet as mx
from mxnet.contrib import onnx as onnx_mxnet
from mxnet import gluon, nd
import numpy as np
    

def model_fn(model_dir):
    """
    Load the onnx model. Called once when hosting service starts.

    :param: model_dir The directory where model files are stored.
    :return: a model
    """
    onnx_path = os.path.join(model_dir, "model.onnx")
    ctx = mx.cpu() # todo: pass into function
    # load onnx model symbol and parameters
    sym, arg_params, aux_params = onnx_mxnet.import_model(onnx_path)
    model_metadata = onnx_mxnet.get_model_metadata(onnx_path)
    # first index is name, second index is shape
    input_names = [inputs[0] for inputs in model_metadata.get('input_tensor_data')]
    input_symbols = [mx.sym.var(i) for i in input_names]
    net = gluon.nn.SymbolBlock(outputs=sym, inputs=input_symbols)
    net_params = net.collect_params()
    # set parameters (on correct context)
    for param in arg_params:
        if param in net_pa

### Endpoint

Now we'll actually create a SageMaker endpoint to call for predictions.

*Note, this step could be replaced by importing the ONNX model into the notebook environment.*

In [8]:
predictor = estimator.deploy(initial_instance_count=1, 
                             instance_type='ml.m4.xlarge', 
                             entry_point='deploy-coach.py')

---------------------------------------------------------------------------------------!

---

## Play 

Let's play our agent.  After running the cell below, just click on one the boxes to make your move.  To restart the game, simply execute the cell again.

*This cell uses the `TicTacToeGame` class from `tic_tac_toe_game.py` script to build an extremely basic tic-tac-toe app within a Jupyter notebook.  The opponents moves are generated by invoking the `predictor` passed at initialization.  Please refer to the code for additional details.*

In [37]:
t = TicTacToeGame(predictor)
t.start()

---

## Wrap Up

In this notebook we trained a reinforcement learning agent to play a simple game of tic-tac-toe, using a custom Gym environment.  It could be built upon to solve other problems or improved by:

- Training for more episodes
- Using a different reinforcement learning algorithm
- Tuning hyperparameters for improved performance
- Or how about a nice game of [global thermonuclear war](https://youtu.be/s93KC4AGKnY?t=41)?

Let's finish by cleaning up our endpoint to prevent any persistent costs.

In [38]:
predictor.delete_endpoint()